# Calorimetry likelihood stopping muons with arrays

In [1]:
%load_ext autoreload
%autoreload 2

import math
from itertools import product
import uproot
import awkward
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from scipy.stats import binned_statistic

import matplotlib
matplotlib.rcParams.update({'font.size': 12})
from calo_likelihood import caloLikelihood
from general_functions import *
from loaders import *
from plotter import *
from categorisation import *
from initializers import *

In [2]:
today = 'calorimetry'
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/pitch_phi/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/pitch_phi/plane0/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/pitch_phi/plane1/
!mkdir -p ~/Dropbox/MicroBooNE/_fig/$today/muons/pitch_phi/plane2/
save_folder = '/home/nic/Dropbox/MicroBooNE/_fig/' + today + '/muons/pitch_phi/'

In [3]:
branches = [
    'run',
    'sub',
    'evt',
    
    'backtracked_pdg',
    'backtracked_e',
    'backtracked_purity',
    'backtracked_completeness',
    'backtracked_overlay_purity',
    'backtracked_end_process',
    
    'backtracked_start_x',
    'backtracked_start_y',
    'backtracked_start_z',
    
    'backtracked_sce_start_x',
    'backtracked_sce_start_y',
    'backtracked_sce_start_z',
    
    'nplanehits_U',
    'nplanehits_V',
    'nplanehits_Y',
    'trk_score',

    'generation',
    'trk_daughters',
    'shr_daughters',

    'trk_sce_start_x',
    'trk_sce_start_y',
    'trk_sce_start_z',

    'trk_sce_end_x',
    'trk_sce_end_y',
    'trk_sce_end_z',
    
    'trk_theta',
    'trk_phi',

    'trk_dir_x',
    'trk_dir_y',
    'trk_dir_z',
    
#     'trk_pid_chipr_y',
#     'trk_pid_chipr_u',
#     'trk_pid_chipr_v',
#     'trk_pid_chimu_y',
#     'trk_bragg_p_y',
#     'trk_bragg_mu_y',

    'trk_len',
    'trk_distance',
    'longest',
    
    'trk_mcs_muon_mom',
    'trk_range_muon_mom',
    
    'is_hit_montecarlo_u',
    'is_hit_montecarlo_v',
    'is_hit_montecarlo_y',
    
    'dqdx_u',
    'dqdx_v',
    'dqdx_y',

#     'dedx_u',
#     'dedx_v',
#     'dedx_y',

    'rr_u',
    'rr_v',
    'rr_y',

    'pitch_u',
    'pitch_v',
    'pitch_y',
    
    'dir_x_u',
    'dir_x_v',
    'dir_x_y',
    
    'dir_y_u',
    'dir_y_v',
    'dir_y_y',
    
    'dir_z_u',
    'dir_z_v',
    'dir_z_y',
]

In [4]:
directory = '/home/nic/Desktop/MicroBooNE/calorimetry_likelihood/v07/'
pot_beam_on = 4.054e+19
n_triggers_on = 8977290.0
n_triggers_off = 33240727.0

file_lists = {}

file_lists['beam_on'] = directory+'beam_on/out_muon_skimmed.root'
file_lists['beam_off'] = directory+'beam_off/out_muon_skimmed.root'
file_lists['bnb_nu'] = directory+'bnb_nu/out_muon_skimmed.root'
file_lists['bnb_dirt'] = directory+'bnb_dirt/out_muon_skimmed.root'

folder = "nuselection"
events_tree_name = "CalorimetryAnalyzer"
pot_tree_name = "SubRun"

In [5]:
arrays, pot = load_data_calo(file_lists, 
                        branches, 
                        folder, 
                        events_tree_name, 
                        pot_tree_name, 
                        labels=['beam_on', 'beam_off', 'bnb_nu', 'bnb_dirt'],
                        lazy=False)

start loading beam_on
start loading beam_off
start loading bnb_nu
start loading bnb_dirt
Done!


In [6]:
scale_factors = compute_scale_factors(pot, pot_beam_on, n_triggers_on, n_triggers_off)

In [7]:
scale_factors

{'beam_on': 1,
 'beam_off': 0.27006900300345416,
 'bnb_nu': 0.03504258914669898,
 'bnb_dirt': 0.12607623142910898}

In [8]:
for name, array in arrays.items():
    print(name)
    initialisation_for_caloriemtry_data_mc(array)

beam_on
beam_off
bnb_nu
bnb_dirt


# Plots

In [9]:
my_plotter = plotter(arrays, scale_factors, pot_beam_on, branch_weights_name='weightSplineTimesTune')
title_base = 'Longest track, residual range > 30 cm\ntrk-score>0.5, start-end fiducial\ntrk-vtx distance < 5 cm, muon-momentum-consistency < 0.5'

In [10]:
parameters = ['pitch', 'abs_theta_yx']
parameters_legend_names = ['pitch', r'$\phi$']
parameters_bin_edges = [[0.3, 0.4, 0.7, 1., 1.5, 30], np.linspace(0, np.pi/2, 7)]

for plane_num, plane in zip([0, 1, 2], ['_u', '_v', '_y']):
    for par0_edges in zip(parameters_bin_edges[0][:-1], parameters_bin_edges[0][1:]):
        for par1_edges in zip(parameters_bin_edges[1][:-1], parameters_bin_edges[1][1:]):
            this_parameters = [par+plane for par in parameters]
            def sel_function(array):
                return (array['first_last_hit_mask'+plane]) &\
                       (array['rr'+plane] > 30) &\
                        ((array[this_parameters[0]]) > par0_edges[0]) &\
                        ((array[this_parameters[0]]) <= par0_edges[1]) &\
                        ((array[this_parameters[1]]) > par1_edges[0]) &\
                        ((array[this_parameters[1]]) <= par1_edges[1])
            label = 'plane {}, {:.2g} < {} < {:.2g} & {:.2g} < {} < {:.2g}'.format(plane_num, par0_edges[0], 
                                                         parameters_legend_names[0],
                                                         par0_edges[1],
                                                         par1_edges[0], 
                                                         parameters_legend_names[1],
                                                         par1_edges[1],)
            save_label = '{:.2f}_{}_{:.2f}_{:.2f}_{}_{:.2f}'.format( 
                                                         par0_edges[0], 
                                                         parameters_legend_names[0],
                                                         par0_edges[1],
                                                         par1_edges[0], 
                                                         parameters_legend_names[1],
                                                         par1_edges[1]).replace('.', '').replace('$', '').replace('\\', '')
            title = title_base + '\n' + label
            out = my_plotter.plot2d(dataset_name='beam_subtracted', 
                                    variables=this_parameters, 
                                    binning=[15, par0_edges[0], par0_edges[1], 15, par1_edges[0], par1_edges[1]], 
                                    additional_selection=sel_function, 
                                    title="Data Beam ON - Beam OFF, residual range > 30 cm\n" + title,
                                    labels=["Hit pitch", r"Hit $\phi$"], 
                                    density=True,
                                    log=True)
            plt.savefig(save_folder + 'plane{}/beam_'.format(plane_num) + save_label+'_beam.png', dpi=300)
            plt.savefig(save_folder + 'plane{}/beam_'.format(plane_num) + save_label+'_beam.pdf')
            plt.close()
            
            out2 = my_plotter.plot2d(dataset_name='bnb_nu', 
                                    variables=this_parameters, 
                                    binning=[15, par0_edges[0], par0_edges[1], 15, par1_edges[0], par1_edges[1]], 
                                    additional_selection=sel_function, 
                                    title="Simulation, residual range > 30 cm\n" + title,
                                    labels=["Hit pitch", r"Hit $\phi$"],  
                                    density=True,
                                    log=True)
            plt.savefig(save_folder + 'plane{}/sim_'.format(plane_num) + save_label+'_sim.png', dpi=300)
            plt.savefig(save_folder + 'plane{}/sim_'.format(plane_num) + save_label+'_sim.pdf')
            plt.close()